## Importing Packages

In [ ]:
import numpy as np
import pandas as pd

## Reading-In and Wrangling Data

In [ ]:
df_spy = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "spy")
df_agg = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "agg")
df_hyg = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "hyg")
df_tlt = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "tlt")
df_gld = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "gld")
df_buffer_010 = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "mqu1bslq")
df_buffer_020 = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "mquslblr")
df_buffer_100 = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "mqu1pplr")
df_sv_hedged_income = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_hedged_income")
df_sv_hedged_balanced = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_hedged_balanced")
df_sv_hedged_enhanced_growth = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_hedged_enhanced_growth")
df_sv_equity_buffer = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_equity_buffer")
df_sv_equity_buffer_growth = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_equity_buffer_growth")
df_spy_corrections = pd.read_csv("..//data/spy_corrections.csv")
df_spy_corrections["start_date"] = pd.to_datetime(df_spy_corrections["start_date"])
df_spy_corrections["end_date"] = pd.to_datetime(df_spy_corrections["end_date"])

In [ ]:
df_px = (
    df_buffer_100
        .merge(df_buffer_010, how="left", on="date")
        .merge(df_buffer_020, how="left", on="date")
        .merge(df_spy, how="left", on="date")
        .merge(df_agg, how="left", on="date")
        .merge(df_hyg, how="left", on="date")
        .merge(df_tlt, how="left", on="date")
        .merge(df_gld, how="left", on="date")
        .merge(df_sv_hedged_income, how="left", on="date")
        .merge(df_sv_hedged_balanced, how="left", on="date")
        .merge(df_sv_hedged_enhanced_growth, how="left", on="date")
        .merge(df_sv_equity_buffer, how="left", on="date")
        .merge(df_sv_equity_buffer_growth, how="left", on="date")
)
df_px.rename(columns={"mqu1pplr":"buffer_100", "mquslblr":"buffer_020", "mqu1bslq":"buffer_010"}, inplace=True)
df_px

,date,buffer_100,buffer_010,buffer_020,spy,agg,hyg,tlt,gld,sv_hedged_income,sv_hedged_balanced,sv_hedged_enhanced_growth,sv_equity_buffer,sv_equity_buffer_growth
0,2006-12-29,1000.0000,1110.43,1099.0969,100.739937,57.753967,NaN,51.180542,63.209999,NaN,NaN,NaN,NaN,NaN
1,2007-01-03,999.8752,1109.92,1097.9915,100.562088,57.875610,NaN,51.545147,62.279999,NaN,NaN,NaN,NaN,NaN
2,2007-01-04,1000.9686,1111.59,1099.8248,100.775497,57.997261,NaN,51.857674,61.650002,NaN,NaN,NaN,NaN,NaN
3,2007-01-05,998.3562,1107.75,1094.9608,99.971687,57.956741,NaN,51.631977,60.169998,NaN,NaN,NaN,NaN,NaN
4,2007-01-08,1000.3551,1110.56,1097.9422,100.434090,57.991474,NaN,51.724541,60.480000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4553,2025-02-05,1540.2790,3921.65,2933.1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4554,2025-02-06,1541.3184,3929.57,2935.2570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4555,2025-02-07,1538.6975,3911.23,2927.5914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4556,2025-02-10,1541.6280,3926.20,2936.4200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Helper Functions

In [ ]:
def max_period_drawdown(asset, date_start, date_end, df_ret):
    col_name = f"equity_{asset}"
    df_temp = df_ret.query("@date_start <= date <= @date_end")[["date", col_name]].copy()
    df_temp["drawdown"] = (df_temp[col_name] / df_temp[col_name].cummax()) - 1
    return df_temp["drawdown"].min()

## Single Backtest

In [ ]:
date_start = '2021-01-15'
date_end = '2024-12-31'

# portfolio = {
#     "spy":0.5,
#     "hyg":0.3,
#     "buffer_100":0.2,
# }
portfolio = {
    "sv_hedged_enhanced_growth":1
}

In [ ]:
def backtester(portfolio=portfolio, date_start=date_start, date_end=date_end, df_px=df_px, df_spy_corrections=df_spy_corrections):
    # filtering for only the dates that we need
    df_px = df_px.query("@date_start <= date & date <= @date_end").copy()
    df_spy_corrections = df_spy_corrections.query("@date_start <= start_date & end_date <= @date_end").copy()

    # isolating weights and assets from portfolio
    assets = []
    weights = []
    for asset, weight in portfolio.items():
        assets.append(asset)
        weights.append(weight)

    # grabbing only the columns needed from df_px
    cols = ["date"] + assets
    df_ret = df_px[cols].copy()

    # calculating component asset daily returns
    for ix_asset in assets:
        ret_col_name = "ret_" + ix_asset
        df_ret[ret_col_name] = df_ret[ix_asset].pct_change()
    df_ret.fillna(0, inplace=True)

    # calculating portfolio daily returns
    cols = []
    for ix_asset in assets:
        cols.append("ret_" + ix_asset)
    df_ret["ret_portfolio"] =  np.sum(np.array(df_ret[cols]) * weights, axis=1)

    # calculating equity curve for components and portfolio
    for ix_asset in assets:
        ret_col_name = "ret_" + ix_asset
        equity_col_name = "equity_" + ix_asset
        df_ret[equity_col_name] = (1 + df_ret[ret_col_name]).cumprod()
    df_ret["equity_portfolio"] = (1 + df_ret["ret_portfolio"]).cumprod()

    # calculating drawdowns for components and portfolio
    for ix_asset in assets:
        equity_col_name = "equity_" + ix_asset
        drawdown_col_name = "drawdown_" + ix_asset
        df_ret[drawdown_col_name] = (df_ret[equity_col_name] / df_ret[equity_col_name].cummax()) - 1
    df_ret["drawdown_portfolio"] = (df_ret["equity_portfolio"] / df_ret["equity_portfolio"].cummax()) - 1

    # calculating cumulative returns
    cumulative_returns = {}
    for ix_asset in assets:
        equity_col_name = "equity_" + ix_asset
        cumulative_returns[ix_asset] = (df_ret[equity_col_name].iloc[-1] - 1)
    cumulative_returns["portfolio"] = df_ret["equity_portfolio"].iloc[-1] - 1

    # calculating annual returns
    annual_returns = {}
    for ix_asset in assets:
        equity_col_name = "equity_" + ix_asset
        annual_returns[ix_asset] = (df_ret[equity_col_name].iloc[-1] ** (252/(len(df_ret) - 1)) - 1)
    annual_returns["portfolio"] = df_ret["equity_portfolio"].iloc[-1] ** (252/(len(df_ret) - 1)) - 1

    # calculating volatilties
    volatility = {}
    for ix_asset in assets:
        ret_col_name = "ret_" + ix_asset
        volatility[ix_asset] = df_ret[ret_col_name][1:].std() * np.sqrt(252)
    volatility["portfolio"] = df_ret["ret_portfolio"][1:].std() * np.sqrt(252)

    # calculating sharpe-ratio
    sharpe = {}
    for ix_asset in assets:
        ret_col_name = "ret_" + ix_asset
        sharpe[ix_asset] = (df_ret[ret_col_name][1:].mean() / df_ret[ret_col_name][1:].std()) * np.sqrt(252)
    sharpe["portfolio"] = (df_ret["ret_portfolio"][1:].mean() / df_ret["ret_portfolio"][1:].std()) * np.sqrt(252)

    # calendar year performance
    df_portfolio = df_ret[["date", "ret_portfolio"]].copy()
    df_portfolio["year"] = df_portfolio["date"].dt.year
    df_annual_performance = df_portfolio.groupby(["year"])[["ret_portfolio"]].agg(lambda x: np.prod(1 + x) - 1).reset_index()
    
    # maximum drawdown
    drawdown_max = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_max[ix_asset] = df_ret[drawdown_col_name].min()
    drawdown_max["portfolio"] =  df_ret["drawdown_portfolio"].min()

    # gfc drawdown
    start = "2007-10-09"
    end = "2009-03-09"
    drawdown_gfc = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_gfc[ix_asset] = max_period_drawdown(asset=ix_asset, date_start=start, date_end=end, df_ret=df_ret)
    drawdown_gfc["portfolio"] = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)

    # pandemic drawdown
    start = "2020-02-19"
    end = "2020-03-23"
    drawdown_pandemic = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_pandemic[ix_asset] = max_period_drawdown(asset=ix_asset, date_start=start, date_end=end, df_ret=df_ret)
    drawdown_pandemic["portfolio"] = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)

    # 2022 drawdown
    start = "2022-01-03"
    end = "2022-10-12"
    drawdown_2022 = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_2022[ix_asset] = max_period_drawdown(asset=ix_asset, date_start=start, date_end=end, df_ret=df_ret)
    drawdown_2022["portfolio"] = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)

    # summer 2024 drawdown
    start = "2024-08-01"
    end = "2024-08-05"
    drawdown_summer_2024 = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_summer_2024[ix_asset] = max_period_drawdown(asset=ix_asset, date_start=start, date_end=end, df_ret=df_ret)
    drawdown_summer_2024["portfolio"] = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)

    # for all market corrections
    drawdowns_portfolio = []
    for ix in df_spy_corrections.index:
        start_date = df_spy_corrections.at[ix, "start_date"]
        end_date = df_spy_corrections.at[ix, "end_date"]
        drawdown_portfolio = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)
        drawdowns_portfolio.append(drawdown_portfolio)
    df_corrections = df_spy_corrections.copy()
    df_corrections["drawdown_portfolio"] = drawdowns_portfolio

    result = {
        "portfolio":portfolio,
        "cumulative_return":cumulative_returns,
        "annual_return":annual_returns,
        "volatility":volatility,
        "sharpe":sharpe,
        "annual_performance":df_annual_performance,
        "drawdown_max":drawdown_max,
        "drawdown_gfc":drawdown_gfc,
        "drawdown_pandemic":drawdown_pandemic,
        "drawdown_2022":drawdown_2022,
        "drawdown_summer_2024":drawdown_summer_2024,
        "spy_corrections":df_corrections,
        "data":df_ret,
    }

    return(result)

In [ ]:
res = backtester()

## Looping Through Portfolios

In [ ]:
df_portfolios = pd.read_excel("..\\data\\portfolios_20250218.xlsx", "Sheet1")
df_portfolios

,name,start_date,end_date,spy,agg,hyg,tlt,gld,buffer_010,buffer_020,buffer_100,sv_hedged_income,sv_hedged_balanced,sv_hedged_enhanced_growth,sv_equity_buffer,sv_equity_buffer_growth
0,spy,2007-04-11,2024-12-31,1.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
1,agg,2007-04-11,2024-12-31,0.00,1.0,0.0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
2,hyg,2007-04-11,2024-12-31,0.00,0.0,1.0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
3,tlt,2007-04-11,2024-12-31,0.00,0.0,0.0,1.00,0.00,0.00,0.00,0.00,0,0,0,0,0
4,gld,2007-04-11,2024-12-31,0.00,0.0,0.0,0.00,1.00,0.00,0.00,0.00,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,growth_model_no_duration,2022-06-30,2024-12-31,0.65,0.0,0.0,0.15,0.10,0.05,0.00,0.05,0,0,0,0,0
306,growth model_3,2022-06-30,2024-12-31,0.70,0.0,0.0,0.15,0.00,0.05,0.05,0.05,0,0,0,0,0
307,growth model_4,2022-06-30,2024-12-31,0.70,0.0,0.0,0.15,0.05,0.00,0.05,0.05,0,0,0,0,0
308,sv_equity_buffer,2022-06-30,2024-12-31,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0


In [ ]:
lst_name = []
lst_date_start = []
lst_date_end = []
lst_weight_spy = []
lst_weight_agg = []
lst_weight_hyg = []
lst_weight_tlt = []
lst_weight_gld = []
lst_weight_buffer_010 = []
lst_weight_buffer_020 = []
lst_weight_buffer_100 = []
lst_weight_sv_hedged_income = []
lst_weight_sv_hedged_balanced = []
lst_weight_sv_hedged_enhanced_growth = []
lst_weight_sv_equity_buffer = []
lst_weight_sv_equity_buffer_growth = []
lst_cumulative_ret = []
lst_annual_ret = []
lst_volatility = []
lst_sharpe = []
lst_annual_performance = []
lst_drawdown_max = []
lst_drawdown_gfc = []
lst_drawdown_pandemic = []
lst_drawdown_2022 = []
lst_drawdown_summer_2024 = []
for _ , row in df_portfolios.iterrows():
    name = row["name"]
    date_start = row["start_date"]
    date_end = row["end_date"]

    # generating portfolio
    portfolio = {}
    weight_spy = row["spy"]
    if weight_spy > 0:
        portfolio["spy"] = weight_spy
    
    weight_agg = row["agg"]
    if weight_agg > 0:
        portfolio["agg"] = weight_agg
        
    weight_hyg = row["hyg"]
    if weight_hyg > 0:
        portfolio["hyg"] = weight_hyg

    weight_tlt = row["tlt"]
    if weight_tlt > 0:
        portfolio["tlt"] = weight_tlt

    weight_gld = row["gld"]
    if weight_gld > 0:
        portfolio["gld"] = weight_gld
        
    weight_buffer_010 = row["buffer_010"]
    if weight_buffer_010 > 0:
        portfolio["buffer_010"] = weight_buffer_010
        
    weight_buffer_020 = row["buffer_020"]
    if weight_buffer_020 > 0:
        portfolio["buffer_020"] = weight_buffer_020
        
    weight_buffer_100 = row["buffer_100"]
    if weight_buffer_100 > 0:
        portfolio["buffer_100"] = weight_buffer_100

    weight_sv_hedged_income = row["sv_hedged_income"]
    if weight_sv_hedged_income > 0:
        portfolio["sv_hedged_income"] = weight_sv_hedged_income

    weight_sv_hedged_balanced = row["sv_hedged_balanced"]
    if weight_sv_hedged_balanced > 0:
        portfolio["sv_hedged_balanced"] = weight_sv_hedged_balanced

    weight_sv_hedged_enhanced_growth = row["sv_hedged_enhanced_growth"]
    if weight_sv_hedged_enhanced_growth > 0:
        portfolio["sv_hedged_enhanced_growth"] = weight_sv_hedged_enhanced_growth

    weight_sv_equity_buffer = row["sv_equity_buffer"]
    if weight_sv_equity_buffer > 0:
        portfolio["sv_equity_buffer"] = weight_sv_equity_buffer

    weight_sv_equity_buffer_growth = row["sv_equity_buffer_growth"]
    if weight_sv_equity_buffer_growth > 0:
        portfolio["sv_equity_buffer_growth"] = weight_sv_equity_buffer_growth

    # running backtest for strategy
    res = backtester(portfolio=portfolio, date_start=date_start, date_end=date_end, df_px=df_px, df_spy_corrections=df_spy_corrections)

    # creating annual performance DataFrame
    df_ap = res["annual_performance"].copy()
    df_ap["strategy"] = name
    df_ap["start"] = date_start
    df_ap["end"] = date_end
    df_ap["spy"] = weight_spy
    df_ap["agg"] = weight_agg
    df_ap["hyg"] = weight_hyg
    df_ap["tlt"] = weight_tlt
    df_ap["gld"] = weight_gld
    df_ap["buffer_010"] = weight_buffer_010
    df_ap["buffer_020"] = weight_buffer_020
    df_ap["buffer_100"] = weight_buffer_100
    df_ap["sv_hedged_income"] = weight_sv_hedged_income
    df_ap["sv_hedged_balanced"] = weight_sv_hedged_balanced
    df_ap["sv_hedged_enhanced_growth"] = weight_sv_hedged_enhanced_growth
    df_ap["sv_equity_buffer"] = weight_sv_equity_buffer
    df_ap["sv_equity_buffer_growth"] = weight_sv_equity_buffer_growth
    lst_annual_performance.append(df_ap)
    
    # saving results to lists, which will be used to create a final results DataFrame
    lst_name.append(name)
    lst_date_start.append(date_start)
    lst_date_end.append(date_end)
    lst_weight_spy.append(weight_spy)
    lst_weight_agg.append(weight_agg)
    lst_weight_hyg.append(weight_hyg)
    lst_weight_tlt.append(weight_tlt)
    lst_weight_gld.append(weight_gld)
    lst_weight_buffer_010.append(weight_buffer_010)
    lst_weight_buffer_020.append(weight_buffer_020)
    lst_weight_buffer_100.append(weight_buffer_100)
    lst_weight_sv_hedged_income.append(weight_sv_hedged_income)
    lst_weight_sv_hedged_balanced.append(weight_sv_hedged_balanced)
    lst_weight_sv_hedged_enhanced_growth.append(weight_sv_hedged_enhanced_growth)
    lst_weight_sv_equity_buffer.append(weight_sv_equity_buffer)
    lst_weight_sv_equity_buffer_growth.append(weight_sv_equity_buffer_growth)
    lst_cumulative_ret.append(res["cumulative_return"]["portfolio"])
    lst_annual_ret.append(res["annual_return"]["portfolio"])
    lst_volatility.append(res["volatility"]["portfolio"])
    lst_sharpe.append(res["sharpe"]["portfolio"])
    lst_drawdown_max.append(res["drawdown_max"]["portfolio"])
    lst_drawdown_gfc.append(res["drawdown_gfc"]["portfolio"])
    lst_drawdown_pandemic.append(res["drawdown_pandemic"]["portfolio"])
    lst_drawdown_2022.append(res["drawdown_2022"]["portfolio"])
    lst_drawdown_summer_2024.append(res["drawdown_summer_2024"]["portfolio"])
    
    print(portfolio, date_start, date_end)

# creating results DataFrame
df_backtest_result = pd.DataFrame({
    "strategy":lst_name,
    "start":lst_date_start,
    "end":lst_date_end,
    "spy":lst_weight_spy,
    "agg":lst_weight_agg,
    "hyg":lst_weight_hyg,
    "tlt":lst_weight_tlt,
    "gld":lst_weight_gld,
    "buffer_010":lst_weight_buffer_010,
    "buffer_020":lst_weight_buffer_020,
    "buffer_100":lst_weight_buffer_100,
    "sv_hedged_income":lst_weight_sv_hedged_income,
    "sv_hedged_balanced":lst_weight_sv_hedged_balanced,
    "sv_hedged_enhanced_growth":lst_weight_sv_hedged_enhanced_growth,
    "sv_equity_buffer":lst_weight_sv_equity_buffer,
    "sv_equity_buffer_growth":lst_weight_sv_equity_buffer_growth,
    "cumulative_ret":lst_cumulative_ret,
    "annual_ret":lst_annual_ret,
    "volatility":lst_volatility,
    "sharpe":lst_sharpe,
    "drawdown_max":lst_drawdown_max,
    "drawdown_gfc":lst_drawdown_gfc,
    "drawdown_pandemic":lst_drawdown_pandemic,
    "drawdown_2022":lst_drawdown_2022,
    "drawdown_summer_2024":lst_drawdown_summer_2024,
})
df_backtest_result

{'spy': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'agg': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'hyg': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'tlt': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'gld': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'buffer_010': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'buffer_020': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'buffer_100': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.5, 'agg': 0.5} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.6, 'agg': 0.4} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.7, 'agg': 0.3} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.8, 'agg': 0.2} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.5, 'hyg': 0.5} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.6, 'hyg': 0.4} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.7, 'hyg': 0.3} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.8, 'hyg': 0.2} 2007-04-11 00:00:00 2024-12-31 00:00:00
{'spy': 0.4, 'agg': 0.4, 'buffer_01

,strategy,start,end,spy,agg,hyg,tlt,gld,buffer_010,buffer_020,...,sv_equity_buffer_growth,cumulative_ret,annual_ret,volatility,sharpe,drawdown_max,drawdown_gfc,drawdown_pandemic,drawdown_2022,drawdown_summer_2024
0,spy,2007-04-11,2024-12-31,1.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0,4.698201,0.103270,0.198669,0.594232,-0.551894,-0.551894,-0.337173,-0.244964,-0.047200
1,agg,2007-04-11,2024-12-31,0.00,1.0,0.0,0.00,0.00,0.00,0.00,...,0,0.653881,0.028822,0.054771,0.546328,-0.184329,-0.128353,-0.095792,-0.145087,-0.000797
2,hyg,2007-04-11,2024-12-31,0.00,0.0,1.0,0.00,0.00,0.00,0.00,...,0,1.284977,0.047776,0.112376,0.471367,-0.342465,-0.341999,-0.220287,-0.155192,-0.009492
3,tlt,2007-04-11,2024-12-31,0.00,0.0,0.0,1.00,0.00,0.00,0.00,...,0,0.701072,0.030459,0.154400,0.271482,-0.483512,-0.161931,-0.157277,-0.299897,0.000000
4,gld,2007-04-11,2024-12-31,0.00,0.0,0.0,0.00,1.00,0.00,0.00,...,0,2.609571,0.075185,0.173417,0.504863,-0.455550,-0.294141,-0.125277,-0.210328,-0.014572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,growth_model_no_duration,2022-06-30,2024-12-31,0.65,0.0,0.0,0.15,0.10,0.05,0.00,...,0,0.426434,0.152919,0.117515,1.269808,-0.140754,NaN,NaN,-0.138654,-0.028601
306,growth model_3,2022-06-30,2024-12-31,0.70,0.0,0.0,0.15,0.00,0.05,0.05,...,0,0.426427,0.152917,0.122546,1.222566,-0.144230,NaN,NaN,-0.143993,-0.030457
307,growth model_4,2022-06-30,2024-12-31,0.70,0.0,0.0,0.15,0.05,0.00,0.05,...,0,0.431028,0.154405,0.121311,1.244422,-0.143890,NaN,NaN,-0.142730,-0.029816
308,sv_equity_buffer,2022-06-30,2024-12-31,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0,0.302513,0.111695,0.065159,1.657901,-0.069329,NaN,NaN,-0.069329,-0.019957


In [ ]:
df_annual_performance = pd.concat(lst_annual_performance)
df_annual_performance

,year,ret_portfolio,strategy,start,end,spy,agg,hyg,tlt,gld,buffer_010,buffer_020,buffer_100,sv_hedged_income,sv_hedged_balanced,sv_hedged_enhanced_growth,sv_equity_buffer,sv_equity_buffer_growth
0,2007,0.029855,spy,2007-04-11,2024-12-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
1,2008,-0.367950,spy,2007-04-11,2024-12-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
2,2009,0.263518,spy,2007-04-11,2024-12-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
3,2010,0.150562,spy,2007-04-11,2024-12-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
4,2011,0.018950,spy,2007-04-11,2024-12-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2023,0.140715,sv_equity_buffer,2022-06-30,2024-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
2,2024,0.121645,sv_equity_buffer,2022-06-30,2024-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
0,2022,0.022959,sv_equity_buffer_growth,2022-06-30,2024-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
1,2023,0.177437,sv_equity_buffer_growth,2022-06-30,2024-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1


## Exporting to Excel

In [ ]:
with pd.ExcelWriter("backtest.xlsx", engine="openpyxl", mode="w", datetime_format="D/M/YYYY") as writer: 
    df_backtest_result.to_excel(writer, sheet_name="results", index=False, startrow=0, startcol=0)
    df_annual_performance.to_excel(writer, sheet_name="annual_performance", index=False, startrow=0, startcol=0)